# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [220]:
client_key = 'redacted_for_public_repo'
client_secret =  'redacted_for_public_repo'

In [221]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [222]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [223]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [224]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAH%2FwewEAAAAA%2FouhakpNM9lpEF6ZF2JLmaWNIHk%3DSFLBCLbJn41P4JHmbct1QrrZQ1cls41BeTVA9bNpnc4GnkSmUk'}

In [225]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [226]:
max_tweets=100
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users =[]

In [227]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [228]:
climate_tweets = response.json()


In [229]:
len(climate_tweets['data'])

100

In [230]:
tweets +=climate_tweets['data']


In [231]:

ids = [tw['id'] for tw in climate_tweets['data']]
    

In [232]:
min_id = min(ids)

In [233]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [234]:
parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'
}

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['data']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users.extend(climate_tweets['includes']['users'])
    parameters['until_id'] = min_id - 1
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

In [235]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 100


# Let's print some tweets

In [236]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [237]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '1181952375399092225',
  'conversation_id': '1549572208539959301',
  'created_at': '2022-07-20T01:49:38.000Z',
  'entities': {'annotations': [{'end': 52,
                                'normalized_text': '117th Congress',
                                'probability': 0.4121,
                                'start': 39,
                                'type': 'Organization'}],
               'urls': [{'display_url': 'twitter.com/MirandaRSuarez…',
                         'end': 298,
                         'expanded_url': 'https://twitter.com/MirandaRSuarez/status/1549507048177532930',
                         'start': 275,
                         'url': 'https://t.co/7kLdKfjeZc'}]},
  'id': '1549572208539959301',
  'lang': 'en',
  'public_metrics': {'like_count': 0,
                     'quote_count': 0,
                     'reply_count': 0,
                     'retweet_count': 0},
  'referenced_tweets': [{'id': '1549507048177532930', 'type': 'quoted'}],
  'reply_s

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [238]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

There are 139 elected officials in the 117th Congress, who refuse to acknowledge the scientific evidence of human-caused climate change. 

That's not the worst of it, even though they are a minority they obstruct all efforts to pass legislation to help mitigate the problem. https://t.co/7kLdKfjeZc

RT @Kit_Yates_Maths: But did thy die of climate change or just with climate change?

RT @bananafitz: International news is like, “more mass death due to global warming and climate change” and local news is like, “bike lanes:…

Its hot in Europe and the fake news is trying to spin it into  that it is due to climate change. 20 years ago the U.N. predicted a "DISASTER if global warming not checked" I heard AOC say we have 12 years left. But buying fossil fuels from the Saudi's is still okay?



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [239]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['entities', 'source', 'referenced_tweets', 'id', 'text', 'created_at', 'reply_settings', 'lang', 'public_metrics', 'conversation_id', 'author_id'])
11


In [240]:
import pandas as pd
import copy

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [241]:
def tweet_generator(tweets):
    for tweet in tweets:
        try:
            modified_tweet = copy.deepcopy(tweet)
            modified_tweet['user_id'] = modified_tweet.pop('author_id')
            modified_tweet['referenced_tweets_ids'] = [ d['id'] for d in modified_tweet['referenced_tweets'] if 'id' in d]
            modified_tweet.pop('referenced_tweets')
            modified_tweet['retweet_count'] = modified_tweet['public_metrics']['retweet_count']
            yield modified_tweet     
        except KeyError as e:
            continue

In [242]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'source': 'Twitter Web App',
  'entities': {'annotations': [{'start': 39,
     'end': 52,
     'probability': 0.4121,
     'type': 'Organization',
     'normalized_text': '117th Congress'}],
   'urls': [{'start': 275,
     'end': 298,
     'url': 'https://t.co/7kLdKfjeZc',
     'expanded_url': 'https://twitter.com/MirandaRSuarez/status/1549507048177532930',
     'display_url': 'twitter.com/MirandaRSuarez…'}]},
  'id': '1549572208539959301',
  'text': "There are 139 elected officials in the 117th Congress, who refuse to acknowledge the scientific evidence of human-caused climate change. \n\nThat's not the worst of it, even though they are a minority they obstruct all efforts to pass legislation to help mitigate the problem. https://t.co/7kLdKfjeZc",
  'created_at': '2022-07-20T01:49:38.000Z',
  'reply_settings': 'everyone',
  'lang': 'en',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'conversation_id': '1549572208539959301',

In [243]:
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [244]:
climate_feed_df

,source,entities,id,text,created_at,reply_settings,lang,public_metrics,conversation_id,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id
0,Twitter Web App,"{'annotations': [{'start': 39, 'end': 52, 'pro...",1549572208539959301,There are 139 elected officials in the 117th C...,2022-07-20T01:49:38.000Z,everyone,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1549572208539959301,1181952375399092225,[1549507048177532930],0,NaN
1,Twitter for Android,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",1549572206233096194,RT @Kit_Yates_Maths: But did thy die of climat...,2022-07-20T01:49:38.000Z,everyone,en,"{'retweet_count': 760, 'reply_count': 0, 'like...",1549572206233096194,1345215509705478146,[1549442694560337920],760,NaN
2,Twitter for Android,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1549572200868593664,"RT @bananafitz: International news is like, “m...",2022-07-20T01:49:37.000Z,everyone,en,"{'retweet_count': 2334, 'reply_count': 0, 'lik...",1549572200868593664,809985698,[1549391709410365443],2334,NaN
3,Twitter Web App,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",1549572198783787009,@Justsay43328336 @PointlessSword @realRussellD...,2022-07-20T01:49:36.000Z,everyone,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1549507568032178176,1500283381036191746,[1549562609262895109],0,1319039866462752770
4,Twitter Web App,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",1549572197005631496,"RT @lavern_spicer: If CLIMATE CHANGE is real, ...",2022-07-20T01:49:36.000Z,everyone,en,"{'retweet_count': 426, 'reply_count': 0, 'like...",1549572197005631496,1522344926100471811,[1549531680465162244],426,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Twitter for Android,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1549571960744837120,RT @cpsavesoil: Burj Khalifa Lights Up In #Sav...,2022-07-20T01:48:39.000Z,everyone,en,"{'retweet_count': 306, 'reply_count': 0, 'like...",1549571960744837120,2788001185,[1549283706388750337],306,NaN
83,Twitter for iPhone,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",1549571951504625669,RT @thetimes: Prince Harry's keynote speech at...,2022-07-20T01:48:37.000Z,everyone,en,"{'retweet_count': 788, 'reply_count': 0, 'like...",1549571951504625669,903564476779298816,[1549284378852958208],788,NaN
84,Twitter for Android,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",1549571950674051078,"RT @ClaireRWong: Guys, I think I’ve solved it....",2022-07-20T01:48:37.000Z,everyone,en,"{'retweet_count': 12253, 'reply_count': 0, 'li...",1549571950674051078,870413586035855360,[1548767909039816709],12253,NaN
85,Twitter for iPhone,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1549571946727489536,RT @BronwynHill1: Any former member of the gov...,2022-07-20T01:48:36.000Z,everyone,en,"{'retweet_count': 30, 'reply_count': 0, 'like_...",1549571946727489536,2411313362,[1549561676865089536],30,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have correct data types. What should be correct datatype?

In [245]:
# write code here datatype listing
climate_feed_df.dtypes


source                   object
entities                 object
id                       object
text                     object
created_at               object
reply_settings           object
lang                     object
public_metrics           object
conversation_id          object
user_id                  object
referenced_tweets_ids    object
retweet_count             int64
in_reply_to_user_id      object
dtype: object

# Write columns with wrong datatype and mention correct data type too.

source should be string
entities should be dict
text should be string
public_metrics should be dict
id should be int64
conversation_id should be int64
reply_settings should be string
lang should be string
created_at should be datetime
user_id should be int64
referenced_tweets_ids should be int64 (or perhaps list if it's a list of ids)
in_reply_to_user_id should be int64

# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [246]:
def tweet_user_generator(tweets):
    for user in climate_tweets['includes']['users']:
        try:
            modified_user = copy.deepcopy(user)
            yield modified_user
        except KeyError as e:
            continue


In [247]:
user_df = pd.DataFrame(tweet_user_generator(climate_tweets))

In [248]:
user_df

,location,username,profile_image_url,protected,created_at,name,url,description,id,public_metrics,verified,pinned_tweet_id,entities
0,"Texas, USA",chilberg11,https://pbs.twimg.com/profile_images/149796300...,False,2019-10-09T15:20:59.000Z,Carl Hilberg,,"If you see something you agree with, feel free...",1181952375399092225,"{'followers_count': 330, 'following_count': 63...",False,NaN,NaN
1,NaN,Skiingmom2,https://pbs.twimg.com/profile_images/154118563...,False,2021-01-02T03:49:18.000Z,Skiingmom🙋‍♀️🎿❄🇨🇦🇦🇹🍻,,"Love my family, skiing & Austria. Maybe not al...",1345215509705478146,"{'followers_count': 274, 'following_count': 71...",False,NaN,NaN
2,the depths of hell,GenericName76,https://pbs.twimg.com/profile_images/152262270...,False,2012-09-08T02:03:51.000Z,Emilio,,I really just tweet shit\n@ewmelendezart #Mama...,809985698,"{'followers_count': 588, 'following_count': 55...",False,1443313198787145739,"{'description': {'hashtags': [{'start': 40, 'e..."
3,NaN,decidealgo,https://pbs.twimg.com/profile_images/129322420...,False,2009-07-11T14:56:49.000Z,"Eugene, Or @decidealgo",,Jesus is Lord!,55847265,"{'followers_count': 1756, 'following_count': 2...",False,NaN,NaN
4,NaN,brushb0t2,https://pbs.twimg.com/profile_images/150031978...,False,2022-03-06T01:33:36.000Z,Hi I'm Dave,,FosterToAdopt. Climate. Solar. Evs. Nuclear. J...,1500283381036191746,"{'followers_count': 32, 'following_count': 261...",False,1508313011177414656,"{'description': {'hashtags': [{'start': 94, 'e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,gopalmarella97,https://pbs.twimg.com/profile_images/154346816...,False,2014-09-03T13:22:29.000Z,gopal #Savesoil,https://t.co/zr2gJrCOb3,"If you really love someone, if you really love...",2788001185,"{'followers_count': 26, 'following_count': 201...",False,1530816939572236289,"{'url': {'urls': [{'start': 0, 'end': 23, 'url..."
90,NaN,RachelPopp7,https://pbs.twimg.com/profile_images/119047451...,False,2017-09-01T10:25:33.000Z,Toasters for Warren 🗽,https://t.co/JDktTNq8P8,Lover of all things Elizabeth Warren.,903564476779298816,"{'followers_count': 2452, 'following_count': 4...",False,1228187584133648384,"{'url': {'urls': [{'start': 0, 'end': 23, 'url..."
91,Canada,Fred_Langlois_,https://pbs.twimg.com/profile_images/126567206...,False,2017-06-01T22:55:45.000Z,Fred Langlois,,Science. Climate justice. Fluffy dogs. Organiz...,870413586035855360,"{'followers_count': 3389, 'following_count': 2...",False,1164601178820399104,"{'description': {'mentions': [{'start': 55, 'e..."
92,Port Macquarie,lorraindeer,https://pbs.twimg.com/profile_images/561026440...,False,2014-03-25T17:13:19.000Z,💜 Lorraine Withers,,Living on Biripi land. Opinions are my own.,2411313362,"{'followers_count': 2044, 'following_count': 9...",False,1083855148915408896,NaN


# Q5(1 point) List datatypes of columns using user_df. Describe which attributes doesn't have correct data types. What should be correct datatype for each case?

In [249]:
# write code here for datatype listing
user_df.dtypes


location             object
username             object
profile_image_url    object
protected              bool
created_at           object
name                 object
url                  object
description          object
id                   object
public_metrics       object
verified               bool
pinned_tweet_id      object
entities             object
dtype: object

# write columns with wrong datatype and mention correct data type too.

profile_image_url should be string
entities should be dict
name should be string
location should be string
id should be int64
public_metrics should be dict
url should be string
username should be string
created_at should be datetime
description should be string
pinned_tweet_id should be int64

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.

In [250]:
retweeted_status_df = climate_feed_df[['id', 'retweet_count']]
climate_feed_df

,source,entities,id,text,created_at,reply_settings,lang,public_metrics,conversation_id,user_id,referenced_tweets_ids,retweet_count,in_reply_to_user_id
0,Twitter Web App,"{'annotations': [{'start': 39, 'end': 52, 'pro...",1549572208539959301,There are 139 elected officials in the 117th C...,2022-07-20T01:49:38.000Z,everyone,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1549572208539959301,1181952375399092225,[1549507048177532930],0,NaN
1,Twitter for Android,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",1549572206233096194,RT @Kit_Yates_Maths: But did thy die of climat...,2022-07-20T01:49:38.000Z,everyone,en,"{'retweet_count': 760, 'reply_count': 0, 'like...",1549572206233096194,1345215509705478146,[1549442694560337920],760,NaN
2,Twitter for Android,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1549572200868593664,"RT @bananafitz: International news is like, “m...",2022-07-20T01:49:37.000Z,everyone,en,"{'retweet_count': 2334, 'reply_count': 0, 'lik...",1549572200868593664,809985698,[1549391709410365443],2334,NaN
3,Twitter Web App,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",1549572198783787009,@Justsay43328336 @PointlessSword @realRussellD...,2022-07-20T01:49:36.000Z,everyone,en,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1549507568032178176,1500283381036191746,[1549562609262895109],0,1319039866462752770
4,Twitter Web App,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",1549572197005631496,"RT @lavern_spicer: If CLIMATE CHANGE is real, ...",2022-07-20T01:49:36.000Z,everyone,en,"{'retweet_count': 426, 'reply_count': 0, 'like...",1549572197005631496,1522344926100471811,[1549531680465162244],426,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Twitter for Android,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1549571960744837120,RT @cpsavesoil: Burj Khalifa Lights Up In #Sav...,2022-07-20T01:48:39.000Z,everyone,en,"{'retweet_count': 306, 'reply_count': 0, 'like...",1549571960744837120,2788001185,[1549283706388750337],306,NaN
83,Twitter for iPhone,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",1549571951504625669,RT @thetimes: Prince Harry's keynote speech at...,2022-07-20T01:48:37.000Z,everyone,en,"{'retweet_count': 788, 'reply_count': 0, 'like...",1549571951504625669,903564476779298816,[1549284378852958208],788,NaN
84,Twitter for Android,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",1549571950674051078,"RT @ClaireRWong: Guys, I think I’ve solved it....",2022-07-20T01:48:37.000Z,everyone,en,"{'retweet_count': 12253, 'reply_count': 0, 'li...",1549571950674051078,870413586035855360,[1548767909039816709],12253,NaN
85,Twitter for iPhone,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",1549571946727489536,RT @BronwynHill1: Any former member of the gov...,2022-07-20T01:48:36.000Z,everyone,en,"{'retweet_count': 30, 'reply_count': 0, 'like_...",1549571946727489536,2411313362,[1549561676865089536],30,NaN


In [251]:
retweeted_status_df

,id,retweet_count
0,1549572208539959301,0
1,1549572206233096194,760
2,1549572200868593664,2334
3,1549572198783787009,0
4,1549572197005631496,426
...,...,...
82,1549571960744837120,306
83,1549571951504625669,788
84,1549571950674051078,12253
85,1549571946727489536,30


# Q7 (1 point) Write code to find number of rows in retweeted_status_df that were not retweeted. print number of such rows.

In [252]:
retweeted_status_df[ retweeted_status_df['retweet_count'] == 0]['id'].count()

20